In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

import openturns as ot
import openturns.viewer as viewer
ot.Log.Show(ot.Log.NONE)

In [ ]:
# Import local classes
# Import local classes
import sys
sys.path.append(r'../../src/')

import bivariate.class_copula

### 1. Wind set-up
$$S = C_2 * \frac{u^2}{g*d}*F$$

In [ ]:
def wind_set_up(u,d,F=3000, C2 = 4.0e-6):
    """
    This function calculates the wind set-up from the wind speed and average depth

    Parameters
    ----------
    u : float
        [m/s] Wind speed
    d : float
        [m] Average depth over fetch length
    F : float
        [m] Fetch length
    C2 : float, optional
        [-] Empirical constant, between 3.5e-6 and 4.0e-6, default is 4.0e-6
    
    Returns
    -------
    S : float
        [m] Wind set-up at toe of the dike
    """

    g = 9.81
    S = C2 * (u**2)/(g*d) * F  
    return S 

### 2. Run-up level
$$Ru_{2\%} = 8H_s*tan(\alpha)$$

In [ ]:
def run_up_level(Hs,alpha):
    import numpy as np
    """
    This function calculates the run-up level from the significant wave height

    Parameters
    ----------
    Hs : float
        [m] Significant wave height
    alpha : float
        [degrees] Angle of the slope of the toe of dike
    
    Returns
    -------
    Ru2 : float
        [m] Run-up level, exceeded by 2% of the waves
    """
    # Since numpy works with radians, we need to convert the angle to radians
    alpha_radians = np.radians(alpha)

    Ru2 = 8*Hs*np.tan(alpha_radians)
    return Ru2


### 3. Average water depth
$$d_{average,fetch} = h_{offshore} - Z_{bed}$$

### 4. Water level at dike
$$d_{waterlevel,dike} = d_{average,fetch} + Ru_{2\%} + S $$ 

### B. Determine statistical properties of Random Variables

In [ ]:
#### Add background information!!!!

### B.1. Plotting distributions


In [ ]:
import bivariate.class_emperical

In [ ]:
# Create object for a random variable
object_Hs = bivariate.class_emperical.Emperical_data(data_title='Hs', data_units='m')

# Add a distribution to the object
object_Hs.add_distribution(distribution_name= 'exponential',
                            rv_continous_frozen= st.expon(loc=0, scale=1.01) )

object_Hs.add_distribution(distribution_name= 'normal',
                            rv_continous_frozen= st.norm(loc=1, scale=0.2) )

# Add extreme value distribution
object_Hs.add_distribution(distribution_name= 'genextreme',
                            rv_continous_frozen= st.genextreme(c=-0.1, loc=1, scale=0.2))
# Plot the distributions
object_Hs.plot_distributions(log_scale=True)

### C.0 Define LSF

In [ ]:
def total_water_level_at_dike(Hs, u, d_average=3, alpha=20, F=10000, C2 = 4.0e-6):
    """
    This function calculates the seperate parts that sum up to total water level at the dike

    Parameters
    ----------
    Hs : float
        [m] Significant wave height
    alpha : float
        [degrees] Angle of the slope of the toe of dike
    u : float
        [m/s] Wind speed
    d : float
        [m] Average depth over fetch length
    F : float
        [m] Fetch length
    h_offshore : float
        [m] Offshore water level
    Zbed : float
        [m] Bed level at the toe of the dike
    
    Returns
    -------
    TWL_sample : 3D array
        [m] Total water level at the dike, consisting of run-up level, wind set-up and average water depth
    
    """
    # Calculate wind set-up
    S = wind_set_up(u,d_average,F, C2)

        
    # Calculate run-up level
    Ru2 = run_up_level(Hs,alpha)
    
    # Calculate total water level
    TWL = d_average + Ru2 + S
    return TWL

### C. Sampling with MCS

In [ ]:
# Determine the random variables 
Hs = st.expon(loc=1, scale=0.2)           # Significant wave height
u = st.expon(loc=1, scale=1.0663)           # Wind speed
h_offshore = st.norm(loc=4, scale=0.4)      # Water level offshore
Z_bed = st.norm(loc=1, scale=0.1)         # Average bed level over the fetch length

In [ ]:
# Generate random samples
n = 1000  # Number of samples

# Generate random samples
Hs_samples = Hs.rvs(size = n)
u_samples = u.rvs(size = n)
h_offshore_samples = h_offshore.rvs(size = n)
Z_bed_samples = Z_bed.rvs(size = n)
depth_average_samples = h_offshore_samples - Z_bed_samples

### C.1 Fixate d_average

In [ ]:
# Combine other two sample list into an array
Hs_u_samples = np.array([Hs_samples, u_samples]).T

def LSF_fix_depth(Hs, u):
    TWL = total_water_level_at_dike(Hs, u, d_average=5, alpha=20, F=10000, C2 = 4.0e-6)
    h_max = 10
    function = (TWL - h_max >= 0)
    return function



In [ ]:
LSF_class_fixate_depth = bivariate.class_copula.Region_of_interest(random_samples=Hs_u_samples, labels = ['Hs [m]', 'u [m/s]'])

# Plot the emperical contours of the random samples
LSF_class_fixate_depth.plot_emperical_contours(bandwidth=1, equal_axis=True)

# Assign the function to the class instance
LSF_class_fixate_depth.function = LSF_fix_depth

# Calculate the probability of failure
LSF_class_fixate_depth.inside_function()

# # Plot the points inside the region of interest
LSF_class_fixate_depth.plot_inside_function(equal_axis=True);



### C.2 New fixate Hs

In [ ]:
# Combine other two sample list into an array
d_u_samples = np.array([depth_average_samples, u_samples]).T

def LSF_fix_Hs(u_input, d_average_input):
    TWL = total_water_level_at_dike(Hs = 3, u = u_input, d_average=d_average_input, alpha=20, F=10000, C2 = 4.0e-6)
    h_max = 10
    function = (TWL - h_max >= 0)
    return function



In [ ]:
LSF_class_fixate_Hs = bivariate.class_copula.Region_of_interest(random_samples=Hs_u_samples, labels = ['d_average [m]', 'u [m/s]'])

# Plot the emperical contours of the random samples
LSF_class_fixate_Hs.plot_emperical_contours(bandwidth=1, equal_axis=True)

# Assign the function to the class instance
LSF_class_fixate_Hs.function = LSF_fix_Hs

# Calculate the probability of failure
LSF_class_fixate_Hs.inside_function()

# # Plot the points inside the region of interest
LSF_class_fixate_Hs.plot_inside_function(equal_axis=True);



### C.3 Fixate u

In [ ]:
# Combine other two sample list into an array
Hs_d_samples = np.array([depth_average_samples, Hs_samples]).T

def LSF_fix_u(Hs_input, d_input):
    TWL = total_water_level_at_dike(Hs = Hs_input, u = 3, d_average=d_input, alpha=20, F=10000, C2 = 4.0e-6)
    h_max = 10
    function = (TWL - h_max >= 0)
    return function



In [ ]:
LSF_class_fixate_u = bivariate.class_copula.Region_of_interest(random_samples=Hs_d_samples, labels = ['d_average [m]', 'Hs [m]'])

# Plot the emperical contours of the random samples
LSF_class_fixate_u.plot_emperical_contours(bandwidth=1, equal_axis=True)

# Assign the function to the class instance
LSF_class_fixate_u.function = LSF_fix_u

# Calculate the probability of failure
LSF_class_fixate_u.inside_function()

# # Plot the points inside the region of interest
LSF_class_fixate_u.plot_inside_function(equal_axis=True);



### E. OpenTurns


In [ ]:
# Determine the random variables 
Hs_OT = ot.Exponential(2.0309)     # Significant wave height
u_OT = ot.Exponential(2.0663)      # Wind speed


h_offshore_OT = ot.Normal(2, 0.2)  # Water level offshore
Z_bed_OT = ot.Normal(0.5, 0.1)     # Average bed level over the fetch length
d_average_OT = h_offshore_OT - Z_bed_OT  # Average depth over the fetch length

In [ ]:
# Determine myLSF
def myLSF(x):
    ''' 
    Vectorized limit-state function.

    Arguments:
    x: vector. x=[Hs, u, h_offshore, Z_bed]. 
    '''
    
    
    # Height of the dike, referenced from the bed level
    z_dike = 10


    g = [z_dike - total_water_level_at_dike(Hs = x[0],alpha = 30, u = x[1], F = 3000, d_average = x[2])]
    return g

In [ ]:
from component_reliability import Case_Study_Compontent_Reliability_Tools

In [ ]:
Case_Study_Compontent_Reliability_Tools.run_FORM_and_MCS(Hs_OT, u_OT, d_average_OT, myLSF, mc_size = 10000)